In [2]:
from google.colab import drive
drive.mount('/content/drive')
SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"];

import csv
import glob
import imageio.v3 as iio
import numpy as np
import os
import pandas as pd
import pickle
import random

Mounted at /content/drive


In [3]:
ROOT_FILE_PATH = "/content/drive/My Drive/Colab Notebooks/2023-Machine-Learning-Dataset";

def ReadNpyFile(folderdir, name):
  retdic = {};
  i = 1;
  retdic[0] = name
  for path in glob.glob(folderdir + "/*.npy"):
    retdic[i] = {}
    retdic[i][0] = path.split("/")[-1]
    retdic[i][1] = np.load(path, allow_pickle = True)
    i += 1
  return retdic;

def ReadPngFile(folderdir, feature, table):
  retdic = {};
  i = 0
  retdic[5000] = feature
  for path in glob.glob(folderdir + "/*.png"):
    retdic[i] = {}
    retdic[i][0] = path.split("/")[-1]
    retdic[i][1] = iio.imread(path)
    retdic[i][2] = {}
    retdic[i][3] = table[i]
    for j in range(425):
      retdic[i][2][j] = {}
      for k in range(425):
         retdic[i][1][j][k] = 0.299*retdic[i][1][j][k][0] + 0.587*retdic[i][1][j][k][1] + 0.114*retdic[i][1][j][k][2]
    i += 1
  return retdic;



In [ ]:
#readme
"""
subject 1-8
image[5000]-->tages_feature_name(133 labels)
image[0-4999]-->picture[0]-->name
         picture[1]-->RGB([0-455][0-455]pixel)(valuw:0-255)
         picture[2]-->gray scale
         picture[3]-->tage(133 labels)(0 or 1)
fmir[0-4999]-->[0]-->  left
        [1]--> righgt

mask[0-31]-->[0]-->name(type of mask)
      -[1]-->np_array
"""

In [4]:
from IPython.utils.sysinfo import get_sys_info
class ReadObject:
  subject = None;
  images = None;
  fmri = [];
  fmri_ex = None;
  masks = None;
  image_data = None;
  table = None;

  def get_info(self, subject):
    image_data = pd.read_csv(os.path.join(ROOT_FILE_PATH, "image_infos", f"{subject}_infos_train.csv"))
    self.table = {}
    for i in range(5000):
      j = 2
      self.table[i] = {}
      for feature in image_data.columns:
        if j > 0:
          j -= 1
          continue
        self.table[i] = np.array(int(image_data.loc[i,feature]))
        eat = []
        eat.append(feature)
        outp = np.array(eat)
    return outp

  def load(self, subject, temp, mode = "train"):
    #Mode = "train" | "test"
    if(subject != self.subject):
      self.subject = subject
      self.masks = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject , "roi_masks"), "roi_masks")

    if(mode == "train"):
      self.images = ReadPngFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_images"), temp, self.table);
      self.fmri_ex = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_fmri"))

    elif(mode == "test"):
      self.images = None
      self.fmri_ex = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "test_split", "test_fmri"))


  def splitted_data(self, batch_number = 1): #batch_number = probability of
    training_set = []
    validation_set = []
    for i in range(5000):
        eat = random.random()
        if eat < batch_number:
            training_set.append(self.image[i])
        else:
            validation_set.append(self.image[i])
    training_set = np.array(training_set)
    validation_set =np.array(validation_set)
    return training_set, validation_set

  def read_fmri(self):
    for i in range(5000):
      self.fmir[i] = []
      self.fmri[i][0] = self.fmri_ex[0][1][i]  #left
      self.fmri[i][1] = self.fmri_ex[1][1][i]  #right



In [ ]:
conbined_data = ReadObject()
subject = SUBJ_LIST[0]
temp = conbined_data.get_info(subject)
conbined_data.load(subject, temp)
conbined_data.read_fmri()
data_train, data_valid = conbined_data.splitted_data(0.9)
np.save('/content/drive/My Drive/Colab Notebooks/data_valid.npy', data_valid)
np.save('/content/drive/My Drive/Colab Notebooks/data_train.npy', data_train)